In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from torch import cuda
import torch
from transformers import AdamW
from tqdm import tqdm

from transformers import AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertModel
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import pickle
from scipy.spatial.distance import cosine

import torch
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load your dataset from the CSV file
df = pd.read_csv('/content/drive/MyDrive/implementation/cleaned_dataset.csv', header= 0, encoding= 'unicode_escape')
#df = df.dropna()
print(df)
#df.head()

                                                 Title Category Views  \
0           mastering java arrays  comprehensive guide   ITJava   85K   
1                   java array manipulation techniques   ITJava   88K   
2                      efficient java array operations   ITJava   82K   
3                        java array sorting algorithms   ITJava   86K   
4                       dynamic arrays java  explained   ITJava   84K   
..                                                 ...      ...   ...   
690      sorting algorithms java  predictive analytics   ITJava   98K   
691  java sorting algorithms  internet things ( iot...   ITJava   96K   
692  sorting algorithms java  natural language proc...   ITJava   93K   
693         java sorting algorithms  quantum computing   ITJava   97K   
694     sorting algorithms java  computational biology   ITJava   94K   

                                              Keywords  
0    java arrays mastering comprehensive guide prog...  
1       j

In [ ]:
X = df['Title'].astype(str)
y = df['Keywords']

#X = df['title'].astype(str)
#y = df['tags']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Tokenize and pad the input data for BERT
def tokenize_and_pad(text):
    tokens = tokenizer.encode(text, max_length=128, truncation=True, padding='max_length', return_tensors='pt')
    return tokens

In [ ]:
X_train_tokens = torch.cat([tokenize_and_pad(text) for text in X_train.values])
X_test_tokens = torch.cat([tokenize_and_pad(text) for text in X_test.values])

y_test_tokens = torch.cat([tokenize_and_pad(text) for text in y_test.values])

In [ ]:
# Use BERT to get embeddings
bert_model = BertModel.from_pretrained('bert-base-uncased')
#bert_model = BertModel.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
with torch.no_grad():
    X_train_embeddings = bert_model(X_train_tokens).last_hidden_state.mean(dim=1)
    X_test_embeddings = bert_model(X_test_tokens).last_hidden_state.mean(dim=1)

    y_test_embeddings = bert_model(y_test_tokens).last_hidden_state.mean(dim=1)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
# Combine BERT embeddings with TF-IDF features
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

y_test_tfidf = tfidf_vectorizer.transform(y_test)

# Concatenate BERT embeddings with TF-IDF features
X_train_combined = torch.cat([torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32), X_train_embeddings], dim=1)
X_test_combined = torch.cat([torch.tensor(X_test_tfidf.toarray(), dtype=torch.float32), X_test_embeddings], dim=1)

y_test_combined = torch.cat([torch.tensor(y_test_tfidf.toarray(), dtype=torch.float32), y_test_embeddings], dim=1)

In [ ]:
# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=240, random_state=128)
rf_model.fit(X_train_combined, y_train)

RandomForestClassifier(n_estimators=240, random_state=128)

In [ ]:
# Make predictions
y_pred = rf_model.predict(X_test_combined)

In [ ]:
with torch.no_grad():
    y_pred_embeddings = bert_model(y_test_tokens).last_hidden_state.mean(dim=1)

y_pred_tfidf = tfidf_vectorizer.transform(y_pred)

y_pred_combined = torch.cat([torch.tensor(y_pred_tfidf.toarray(), dtype=torch.float32), y_pred_embeddings], dim=1)

# Evaluation

In [ ]:
def two_vs_two(y_test, preds):

    y_pred_unique = np.unique(preds, axis=0)
    y_test_unique = np.unique(y_test, axis=0)

    points = 0
    total_points = 0
    for i in range(y_pred_unique.shape[0] - 1):
        s_i = y_test_unique[i]
        s_i_pred = y_pred_unique[i]
        for j in range(i+1, y_pred_unique.shape[0]):
            s_j = y_test_unique[j]
            s_j_pred = y_pred_unique[j]

            # Compute cosine distance.
            dsii = cosine(s_i, s_i_pred)
            dsjj = cosine(s_j, s_j_pred)
            dsij = cosine(s_i, s_j_pred)
            dsji = cosine(s_j, s_i_pred)


            if dsii + dsjj <= dsij + dsji:
                points += 1
            total_points += 1

    return points, total_points, points * 1.0 / total_points

In [ ]:
#points, total_points , over_vector_accurancy = two_vs_two(y_test_combined, y_pred_combined)
points, total_points , over_vector_accurancy = two_vs_two(y_test_combined, y_pred_combined)
formatted_number = str("{:.2%}".format(over_vector_accurancy))

print("number of points that 2 vs. 2 scored = " + str(points))
print("total number of points that 2 vs. 2 has counted = " + str(total_points))
print("overall accuracy 2 vs. 2 that gave the score = " + formatted_number)

number of points that 2 vs. 2 scored = 7666
total number of points that 2 vs. 2 has counted = 9591
overall accuracy 2 vs. 2 that gave the score = 79.93%


# Model saving

In [ ]:
 # Save model
with open('/content/drive/MyDrive/implementation/best_model.pkl', 'wb') as f:
     pickle.dump(rf_model, f)

with open('/content/drive/MyDrive/implementation/vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)


# Small prototype

In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Function to predict keywords for a given title
def predict_keywords(title, rf_model, tokenizer, bert_model, tfidf_vectorizer):

    title = title.lower()

    tokens = nltk.word_tokenize(title)
    filtered_tokens = [token for token in tokens if token not in stopwords_set]
    filtered_text = ' '.join(filtered_tokens)

    print(filtered_text)

    # Tokenize and pad the input title for BERT
    title_tokens = tokenizer.encode(filtered_text, max_length=128, truncation=True, padding='max_length', return_tensors='pt')

    # Get BERT embeddings
    with torch.no_grad():
        title_embedding = bert_model(title_tokens).last_hidden_state.mean(dim=1)

    # TF-IDF vectorization
    title_tfidf = tfidf_vectorizer.transform([title])

    # Concatenate BERT embeddings with TF-IDF features
    title_combined = torch.cat([torch.tensor(title_tfidf.toarray(), dtype=torch.float32), title_embedding], dim=1)

    # Make prediction using the trained model
    predicted_keywords = rf_model.predict(title_combined)

    return predicted_keywords

In [ ]:
# Example usage:
user_input_title = "Sorting Algorithms in Java"
predicted_keywords = predict_keywords(user_input_title, rf_model, tokenizer, bert_model, tfidf_vectorizer)

print(f"Predicted Keywords for '{user_input_title}': {predicted_keywords}")

sorting algorithms java
Predicted Keywords for 'Sorting Algorithms in Java': ['java sorting algorithms mastering expertise']


# Update prototype

In [ ]:
import torch
import pickle
from transformers import BertModel, BertTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
with open('/content/drive/MyDrive/implementation/best_model.pkl', 'rb') as f:
    rf_model = pickle.load(f)

In [ ]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')

# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

In [ ]:
def preprocess_titles(titles):
    # Tokenize and pad the input data for BERT
    def tokenize_and_pad(text):
        tokens = tokenizer.encode(text, max_length=128, truncation=True, padding='max_length', return_tensors='pt')
        return tokens

    # Get BERT embeddings
    def get_bert_embeddings(tokens):
        with torch.no_grad():
            embeddings = bert_model(tokens).last_hidden_state.mean(dim=1)
        return embeddings

    # Tokenize and pad titles
    title_tokens = torch.cat([tokenize_and_pad(title) for title in titles])

    # Get BERT embeddings for titles
    title_embeddings = get_bert_embeddings(title_tokens)

    # Get TF-IDF features for titles
    title_tfidf = tfidf_vectorizer.transform(titles)

    # Combine BERT embeddings with TF-IDF features
    title_combined = torch.cat([torch.tensor(title_tfidf.toarray(), dtype=torch.float32), title_embeddings], dim=1)

    return title_combined

In [ ]:
def predict_tag(title):
    # Tokenize the input title
    tokenized_title = tokenizer.encode(title, max_length=128, truncation=True, padding='max_length', return_tensors='pt')

    # Get BERT embeddings for the input title
    with torch.no_grad():
        title_embedding = bert_model(tokenized_title).last_hidden_state.mean(dim=1)

    # Convert the input title to TF-IDF features
    title_tfidf = tfidf_vectorizer.transform([title])

    # Combine BERT embeddings with TF-IDF features
    combined_features = torch.cat([torch.tensor(title_tfidf.toarray(), dtype=torch.float32), title_embedding], dim=1)

    # Make prediction using the loaded Random Forest model
    predicted_tag = rf_model.predict(combined_features)

    return predicted_tag[0]

In [ ]:
# Get user input
user_title = input("Enter the title: ")

# Make prediction
predicted_tag = predict_tag(user_title)
print("Predicted tag for the given title:", predicted_tag)

Enter the title: array in java
Predicted tag for the given title: casting in java, explicit casting, implicit casting, explicit casting in java, implicit casting in java


# Prototype

In [ ]:
with open('/content/drive/MyDrive/implementation/best_model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('/content/drive/MyDrive/implementation/vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [ ]:
# Load BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')